In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time
pd.set_option('display.max_colwidth', -1)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

First we head to http://research.un.org/en/docs/sc/quick/veto where the UN librarians maintain a list of vetoes.

In [2]:
response = requests.get('http://www.un.org/depts/dhl/resguide/scact_veto_table_en.htm')
soup_doc = BeautifulSoup(response.content, 'html.parser')
#print(soup_doc.prettify())

In [3]:
#Before we get started, we also need to specify a bunch of stuff for a selenium driver, whose only purpose is to grab
#the document name.

download_dir = "/Users/max/Lede/Data_Databases/unsc_vetos/final_project/presentation/pdfs/"
options = webdriver.ChromeOptions()

#NB I'm keeping the pdf viewer enable below, as this was the only way I found to grab the file name.
profile = {"plugins.plugins_list": [{"enabled": True, "name": "Chrome PDF Viewer"}],
               "download.default_directory": download_dir , "download.extensions_to_open": "applications/pdf"}
options.add_experimental_option("prefs", profile)

driver = webdriver.Chrome(chrome_options=options)

# Don't run the below unless you have some time to spare
The UN names its documents in a way that I can only access with Selenium, so this one takes a while.

In [6]:
#Sigh. Here we go:
veto_list = []
for cell in soup_doc.findAll('tr')[3:]:
    #print('---------------')
    veto_dict = {}
    veto_dict['date'] = cell.findAll('td')[0].string
    veto_dict['draft'] = cell.findAll('a')[0].string
    veto_dict['draft_url'] = cell.findAll('a')[0].get('href')
    veto_dict['meeting_record'] = cell.findAll('a')[1].string
    #print(cell.findAll('a')[1].string)
    veto_dict['meeting_record_url'] = cell.findAll('a')[1].get('href')
    veto_dict['agenda'] = cell.findAll('td')[3].string
    if cell.findAll('td')[4].next.next == '\n':
        veto_dict['veto_power'] = cell.findAll('td')[4].string
    else:
        veto_dict['veto_power'] = cell.findAll('td')[4].string + cell.findAll('td')[4].next.next + cell.findAll('td')[4].next.next.next
    veto_list.append(veto_dict)
#veto_list

In [7]:
df = pd.DataFrame(veto_list)
df.veto_power.value_counts()

USSR                         90
USA                          59
Russian Federation           14
France UK USA                11
UK USA\n                     9 
UK                           5 
ChinaRussian Federation\n    4 
ChinaRussian Federation      4 
China                        3 
FranceUK USA                 2 
France                       1 
France UK\n                  1 
FranceUK\n                   1 
Name: veto_power, dtype: int64

In [8]:
df.to_csv('raw_scrape.csv', index=False)